# Image Encryption and Decryption Application
---
This Jupyter Notebook demonstrates an image encryption and decryption application using AES encryption.
The application allows the user to encrypt and decrypt image files using a specified password.

## Import Libraries

In [1]:
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os
from PIL import Image
import io

## Define Encryption Function
The `encrypt_image` function encrypts an image file using AES encryption and saves the encrypted file.

In [2]:
def encrypt_image(input_file, output_file, password):
    # Generate a key and nonce from the password using scrypt
    salt = get_random_bytes(16)
    key = scrypt(password, salt, 32, N=2**14, r=8, p=1)
    nonce = get_random_bytes(16)

    # Initialize AES cipher in GCM mode
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    # Read the input image
    with open(input_file, 'rb') as f:
        image_data = f.read()

    # Encrypt the image data
    ciphertext, tag = cipher.encrypt_and_digest(image_data)

    # Write salt, nonce, tag, and ciphertext to the output file
    with open(output_file, 'wb') as f:
        f.write(salt)
        f.write(nonce)
        f.write(tag)
        f.write(ciphertext)

    print(f'Encryption successful. Encrypted image saved at: {output_file}')

## Define Decryption Function
The `decrypt_image` function decrypts an encrypted image file using AES encryption and saves the decrypted image.

In [3]:
def decrypt_image(input_file, output_file, password):
    try:
        # Read salt, nonce, tag, and ciphertext from the input file
        with open(input_file, 'rb') as f:
            salt = f.read(16)
            nonce = f.read(16)
            tag = f.read(16)
            ciphertext = f.read()

        # Generate key from password and salt using scrypt
        key = scrypt(password, salt, 32, N=2**14, r=8, p=1)

        # Initialize AES cipher in GCM mode
        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

        # Decrypt the ciphertext
        decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)

        # Create an image from decrypted data
        decrypted_image = Image.open(io.BytesIO(decrypted_data))

        # Save the decrypted image with a valid image extension
        decrypted_image.save(output_file)

        print(f'Decryption successful. Decrypted image saved at: {output_file}')

    except ValueError:
        print("Decryption failed. The provided key is incorrect. Please try again with the correct key.")

## Define Helper Functions
### Get Operation Choice
The `get_operation_choice` function prompts the user to choose between encryption and decryption.

In [4]:
def get_operation_choice():
    while True:
        choice = input("Enter 'E' for encryption or 'D' for decryption: ").strip().upper()
        if choice == 'E' or choice == 'D':
            return choice
        else:
            print("Invalid choice. Please enter 'E' for encryption or 'D' for decryption.")

### Get Image Path
The `get_image_path` function prompts the user to enter the full path of the image file.

In [5]:
def get_image_path():
    while True:
        image_path = input("Enter the full path of the image file: ").strip()
        if os.path.isfile(image_path):
            return image_path
        else:
            print(f"File not found at '{image_path}'. Please enter a valid image file path.")

## Main Function
The `main` function ties everything together, prompting the user for input and calling the appropriate functions for encryption or decryption.

In [6]:
if __name__ == '__main__':
    # Get operation choice (encryption or decryption)
    operation = get_operation_choice()

    # Get image file path
    image_path = get_image_path()

    # Determine output file paths
    file_directory = os.path.dirname(image_path)
    image_filename = os.path.basename(image_path)
    image_name, image_extension = os.path.splitext(image_filename)

    if operation == 'E':
        output_file = os.path.join(file_directory, image_name + '_encrypted.enc')
    elif operation == 'D':
        output_file = os.path.join(file_directory, image_name + '_decrypted.png')

    # Get password
    password = input("Enter password (must be at least 16 characters long): ").encode('utf-8')  # Convert to bytes

    # Perform chosen operation
    if operation == 'E':
        encrypt_image(image_path, output_file, password)
    elif operation == 'D':
        decrypt_image(image_path, output_file, password)

## Usage Instructions
### Downloading and Running the Code
1. **Install Git:** If you don't have Git installed, download and install it from [git-scm.com](https://git-scm.com/).
2. **Clone the Repository:**
   - Open a terminal or command prompt.
   - Navigate to the directory where you want to clone the repository.
   - Run the following command to clone the repository:
     ```sh
     git clone https://github.com/muhammadalhayani/Cyber-Security-Portfolio.git
     ```
3. **Navigate to the Cloned Repository:**
   ```sh
   cd Cyber-Security-Portfolio/Python-Tools
   ```
4. **Run the Script:**
   - Ensure you have the required libraries installed. You can install them using pip:
     ```sh
     pip install pycryptodome pillow
     ```
   - Run the Python script:
     ```sh
     python Caesar_Cipher.py
     ```
5. **Follow the Prompts:** Enter the required information as prompted by the script to encrypt or decrypt an image.